In [1]:
import os
import sys
import pandas as pd
from tqdm import tqdm
import requests
from sqlalchemy.orm import Session
from dotenv import load_dotenv

load_dotenv()
News_API_KEY = os.environ.get('News_API_KEY')

# modify sys.path for it to contain the main repo path so we can import modules such as below
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from db.db_conn import engine, session_scope, ping_db
from db.models import DimensionOrganization, FactNews

In [17]:
ping_db(engine)

True

In [18]:
def get_organisation(organisation: str):
    if organisation is None:
        raise ValueError('No organisation provided.')
    
    try:
        # Define the API endpoint and parameters
        url = "https://api.thenewsapi.com/v1/news/all"
        params = {
            'api_token': News_API_KEY,
            'language': 'en',
            'search': organisation,
            'published_after': '2024-01-01'
        }

        # Make the GET request
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an error for bad responses
        news_data = response.json().get('data', [])
        news_df = pd.DataFrame(news_data)
        
        return news_df

    except Exception as e:
        print(f"Could not get news for {organisation} due to error: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error
    
def collect_news_data(session):
    news_data = []

    try:
        # Retrieve all organizations
        records = session.query(DimensionOrganization).all()
        for record in tqdm(records, total=len(records), desc="Collecting news data..."):
            organization_name = record.organization_name
            news_df = get_organisation(organization_name)

            # Ensure we have data to insert
            if not news_df.empty:
                for _, row in news_df.iterrows():
                    # Append each news item as a dictionary to the list, including uuid
                    news_data.append({
                        "organization_id": record.organization_id,  # Using organization_id for relationship
                        "uuid": row.get("uuid"),  # Add uuid field here
                        "title": row.get("title"),
                        "description": row.get("description"),
                        "keywords": row.get("keywords"),
                        "snippet": row.get("snippet"),
                        "url": row.get("url"),
                        "image_url": row.get("image_url"),
                        "language": row.get("language"),
                        "published_at": row.get("published_at"),
                        "source": row.get("source"),
                        "categories": row.get("categories"),
                        "relevance_score": row.get("relevance_score", None)
                    })
    
    except Exception as e:
        print(e)
        print("Error occurred while collecting news data.")

    # Convert the list of dictionaries to a DataFrame
    news_df_final = pd.DataFrame(news_data)
    return news_df_final


def add_news_data_to_db(news_df, session: Session):
    try:
        # Iterate over each row in the DataFrame and insert into FactNews table if not a duplicate
        for _, row in news_df.iterrows():
            # Check if the news entry already exists using the uuid
            existing_entry = session.query(FactNews).filter_by(uuid=row.get("uuid")).first()

            # Only add the entry if it does not already exist
            if not existing_entry:
                news_entry = FactNews(
                    organization_id=row["organization_id"],
                    uuid=row.get("uuid"),  # Storing uuid from API
                    title=row.get("title"),
                    description=row.get("description"),
                    keywords=row.get("keywords"),
                    snippet=row.get("snippet"),
                    url=row.get("url"),
                    image_url=row.get("image_url"),
                    language=row.get("language"),
                    published_at=row.get("published_at"),
                    source=row.get("source"),
                    categories=row.get("categories"),
                    relevance_score=row.get("relevance_score", None)
                )
                session.add(news_entry)  # Add each news item to the session

        # Commit all entries at once after adding them to the session
        session.commit()
        print("News data added to FactNews table successfully.")

    except Exception as e:
        session.rollback()  # Rollback in case of error
        print(f"Error occurred while adding news data to database: {e}")

In [19]:
# Collect News Data
with session_scope() as session:
    news_df = collect_news_data(session)

# Display or print the DataFrame for verification
print(news_df.head())

# Populate News to new fact table
with session_scope() as session:
    add_news_data_to_db(news_df, session)

Could not get news for Tilta due to error: 402 Client Error: Payment Required for url: https://api.thenewsapi.com/v1/news/all?api_token=nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX&language=en&search=Tilta&published_after=2024-01-01


Could not get news for ModuGen due to error: 402 Client Error: Payment Required for url: https://api.thenewsapi.com/v1/news/all?api_token=nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX&language=en&search=ModuGen&published_after=2024-01-01


Could not get news for Silmuke due to error: 402 Client Error: Payment Required for url: https://api.thenewsapi.com/v1/news/all?api_token=nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX&language=en&search=Silmuke&published_after=2024-01-01


Could not get news for Ketokamu due to error: 402 Client Error: Payment Required for url: https://api.thenewsapi.com/v1/news/all?api_token=nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX&language=en&search=Ketokamu&published_after=2024-01-01


Could not get news for Cerabyte due to error: 402 Client Error: Payment Required for url: https://api.thenewsapi.com/v1/news/all?api_token=nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX&language=en&search=Cerabyte&published_after=2024-01-01


...

Could not get news for Sunmaxx PVT due to error: 429 Client Error: Too Many Requests for url: https://api.thenewsapi.com/v1/news/all?api_token=nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX&language=en&search=Sunmaxx+PVT&published_after=2024-01-01


Could not get news for &Hamlet due to error: 429 Client Error: Too Many Requests for url: https://api.thenewsapi.com/v1/news/all?api_token=nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX&language=en&search=%26Hamlet&published_after=2024-01-01


Could not get news for eleQtron due to error: 429 Client Error: Too Many Requests for url: https://api.thenewsapi.com/v1/news/all?api_token=nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX&language=en&search=eleQtron&published_after=2024-01-01


Could not get news for AirForestry due to error: 429 Client Error: Too Many Requests for url: https://api.thenewsapi.com/v1/news/all?api_token=nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX&language=en&search=AirForestry&published_after=2024-01-01
   organization_id                                  uuid  \
0                1  457b6e79-6a5f-4a63-a0b0-1ad6a709303a   
1                1  42fbac59-1e22-4f6e-ae41-15d22da97f5f   
2                1  3c77075a-b8f2-4f65-a403-8327a2d31ef1   
3                3  0fec3c09-14c5-49fe-8ede-c811391c638a   
4                3  150228d0-9d0e-4c9b-a691-34ea7d537903   

                                               title  \
0  Inside Flagright: Our Approach to Hiring and T...   
1     Fintech Rundown: A Rapid Review of Weekly News   
2  UK climate fintech Zero secures partners ahead...   
3  Cloud Native Digest: Score accepted as a CNCF ...   
4  AI: A Game-Changer for SREs and Container Orch...   

                                         description  \
0  /

News data added to FactNews table successfully.


In [20]:
news_df.head()

,organization_id,uuid,title,description,keywords,snippet,url,image_url,language,published_at,source,categories,relevance_score
0,1,457b6e79-6a5f-4a63-a0b0-1ad6a709303a,Inside Flagright: Our Approach to Hiring and T...,/PRNewswire-PRWeb/ -- Since the beginning of o...,Flagright,"At Flagright, we believe that a lean, talent-d...",https://www.prweb.com/releases/inside-flagrigh...,https://mma.prnewswire.com/media/2503889/cover...,en,2024-09-14T08:00:00.000000Z,prweb.com,[tech],44.791160
1,1,42fbac59-1e22-4f6e-ae41-15d22da97f5f,Fintech Rundown: A Rapid Review of Weekly News,The week begins with big news on the payment c...,,The week begins with big news on the payment c...,https://finovate.com/fintech-rundown-a-rapid-r...,https://finovate.com/wp-content/uploads/2024/0...,en,2024-02-20T19:25:02.000000Z,finovate.com,[business],20.733753
2,1,3c77075a-b8f2-4f65-a403-8327a2d31ef1,UK climate fintech Zero secures partners ahead...,"Zero, a UK finance app targeting climate-activ...","Finextra, news, online, bank, banking, technol...","Zero, a UK finance app targeting climate-activ...",https://www.finextra.com/newsarticle/44640/uk-...,https://www.finextra.com/finextra-images/top_p...,en,2024-08-28T00:01:00.000000Z,finextra.com,[business],20.349659
3,3,0fec3c09-14c5-49fe-8ede-c811391c638a,Cloud Native Digest: Score accepted as a CNCF ...,Kubermatic KubeOne automates cluster operation...,,Cloud Native Digest: Score accepted as a CNCF ...,https://kubesphere.medium.com/cloud-native-dig...,https://miro.medium.com/v2/resize:fit:1200/1*D...,en,2024-08-12T09:06:09.000000Z,medium.com,[tech],12.997782
4,3,150228d0-9d0e-4c9b-a691-34ea7d537903,AI: A Game-Changer for SREs and Container Orch...,Kubernetes has revolutionized how we manage an...,,AI: A Game-Changer for SREs and Container Orch...,https://medium.com/@levitatingmonkofshambhala/...,https://miro.medium.com/v2/da:true/resize:fit:...,en,2024-08-26T08:54:39.000000Z,medium.com,[tech],12.339506
